### Load Digits Dataset and Create RBF Prototypes

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from PIL import Image
import os
from collections import defaultdict

class DigitsDataset(Dataset):
    def __init__(self, image_folder='./digits updated/', transform=transforms.Compose([transforms.Resize((7, 12)),transforms.ToTensor()])):
        self.image_folder = image_folder
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Load all images and labels from the folder
        for label in range(10):  # Assuming 10 classes (0-9)
            class_folder = os.path.join(image_folder, str(label))
            for img_name in os.listdir(class_folder):
                img_path = os.path.join(class_folder, img_name)
                self.image_paths.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load the image
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        
        # Apply transformation if specified
        if self.transform:
            img = self.transform(img)
            print("transform")

        return img, label
    
# Function to Compute Prototypes
def compute_prototypes_from_images(dataloader):
    class_images = defaultdict(list)

    with torch.no_grad():
        for images, labels in dataloader:
            for img, label in zip(images, labels):
                class_images[label.item()].append(img.squeeze())  # Remove channel dimension

    prototypes = []
    for c in range(10):
        imgs = torch.stack(class_images[c])  # Shape: [N, 7, 12]
        avg_img = imgs.mean(dim=0)           # Shape: [7, 12]
        proto_vec = avg_img.flatten()        # Shape: [84]
        prototypes.append(proto_vec)

    return torch.stack(prototypes)           # Shape: [10, 84]

In [ ]:
dataset = DigitsDataset()
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Compute prototypes
prototypes = compute_prototypes_from_images(data_loader)
print(prototypes.shape)  # Should print: torch.Size([10, 84])

trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform
trainsform

### LeNet5 Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

class LeNet5_S2Layer(nn.Module):
    def __init__(self, num_channels):
        super(LeNet5_S2Layer, self).__init__()
        self.coefficient = nn.Parameter(torch.ones(num_channels))
        self.bias = nn.Parameter(torch.zeros(num_channels))

    def forward(self, x):
        pooled = nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
        pooled = pooled * self.coefficient.view(1, -1, 1, 1)
        pooled = pooled + self.bias.view(1, -1, 1, 1)
        return torch.sigmoid(pooled)

class ScaledTanh(nn.Module):
    def forward(self, x):
        return 1.7159 * torch.tanh(x * 2 / 3)
    
class LeNet5_C3Layer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, connection_table):
        super(LeNet5_C3Layer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.connection_table = connection_table
        
        # Create a convolutional layer with the specified parameters
        self.weight = nn.Parameter(torch.zeros(out_channels, in_channels, kernel_size, kernel_size))
        self.bias = nn.Parameter(torch.zeros(out_channels))
        self.mask = torch.zeros_like(self.weight)

        for out_c, in_list in enumerate(connection_table):
            for in_c in in_list:
                self.mask[out_c, in_c, :, :] = 1.0

        self.reset_parameters()

    def reset_parameters(self):
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
        nn.init.uniform_(self.weight, -2.4 / fan_in, 2.4 / fan_in)
        self.bias.data.fill_(2.4 / fan_in)

    def forward(self, x):
        # Apply the mask before convolution to zero-out unwanted connections
        masked_weight = self.weight * self.mask.to(self.weight.device)
        return F.conv2d(x, masked_weight, self.bias, stride=1)    
    
class RBFLayer(nn.Module):
    def __init__(self, prototype_vectors):
        super(RBFLayer, self).__init__()
        self.prototype_vectors = prototype_vectors

    def forward(self, x):
        x = x.view(x.size(0), -1)

        # Calculate Euclidean distance between input and prototype vectors
        dist = torch.cdist(x, self.prototype_vectors)  # [batch_size, num_classes]

        # Apply the Gaussian kernel (RBF function)
        gamma = 1.0  # You can also learn this parameter or tune it
        rbf_output = torch.exp(-gamma * dist ** 2)

        return rbf_output  # Shape: [batch_size, num_classes]

class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        
        # C1
        self.layer1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.activation1 = ScaledTanh()

        # S2
        self.pool1 = LeNet5_S2Layer(num_channels=6)

        # C3
        connection_table = [
            [0, 1, 2],
            [1, 2, 3],
            [2, 3, 4],
            [3, 4, 5],
            [0, 4, 5],
            [0, 1, 5],
            [0, 1, 2, 3],
            [1, 2, 3, 4],
            [2, 3, 4, 5],
            [0, 3, 4, 5],
            [0, 1, 4, 5],
            [0, 1, 2, 5],
            [0, 1, 3, 4],
            [1, 2, 4, 5],
            [1, 2, 3, 5],
            [0, 1, 2, 3, 4, 5]]

        self.layer2 = LeNet5_C3Layer(in_channels=6, out_channels=16, kernel_size=5, connection_table=connection_table)
        self.activation2 = ScaledTanh()

        # S4
        self.pool2 = LeNet5_S2Layer(num_channels=16)

        # C5
        self.layer3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0)

        # F6
        self.layer4 = nn.Linear(in_features=120, out_features=84)
        self.activation4 = ScaledTanh()

        # Output layer
        self.layer5 = RBFLayer(prototype_vectors=prototypes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation1(x)
        # print(f"After Conv1: {x.shape}")  # Debug print after conv1

        x = self.pool1(x)
        # print(f"After Pool1: {x.shape}")  # Debug print after pooling

        x = self.layer2(x)
        x = self.activation2(x)
        # print(f"After Conv2: {x.shape}")  # Debug print after conv2

        x = self.pool2(x)
        # print(f"After Pool2: {x.shape}")  # Debug print after pooling

        x = self.layer3(x)
        
        # print(x.shape)

        # Flatten the output for the fully connected layer
        x = x.view(x.size(0), -1)
        # print(f"After Flatten: {x.shape}")  # Debug print after flattening

        x = self.layer4(x)
        x = self.activation4(x)
        # print(f"After FC1: {x.shape}")  # Debug print after fc1

        x = self.layer5(x)
        # print(f"Output: {x.shape}")  # Debug print after fc3

        return x

### Load the Training and Testing Dataset from MNIST

In [69]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class CustomImageDataset(Dataset):
    def __init__(self, image_folder, label_file, transform=None):
        self.image_folder = image_folder
        self.label_file = label_file
        self.transform = transform

        # Load image filenames from the folder
        self.image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) if fname.endswith('.png')]
        self.labels = self._load_labels(label_file)

    def _load_labels(self, label_file):
        labels = []
        with open(label_file, 'r') as f:
            labels = [int(line.strip()) for line in f.readlines()]
        return labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # Load image using PIL
        image = Image.open(image_path).convert('RGB')  # Assuming images are RGB

        # Apply transformations if any
        if self.transform:
            image = self.transform(image)

        return image, label

In [70]:
# Training and testing the model on MNIST data:

from mnist import MNIST

# Define relevant variables
batch_size = 1
num_classes = 10
learning_rate = 0.001
num_epochs = 30

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

# Load training and testing datasets
train_image_folder = './data/train'
test_image_folder = './data/test'
train_label_file = './data/train_label.txt'
test_label_file = './data/test_label.txt'
train_dataset = CustomImageDataset(image_folder=train_image_folder, label_file=train_label_file, transform=transform)
test_dataset = CustomImageDataset(image_folder=test_image_folder, label_file=test_label_file, transform=transform)


# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("data loaded")

data loaded


In [71]:
model = LeNet5(num_classes=num_classes).to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomLoss(nn.Module):
    def __init__(self, j=0.01):
        super(CustomLoss, self).__init__()
        self.j = torch.tensor(j)

    def forward(self, logits, labels):
        num_classes = logits.size(1) # 10 classes

        # Convert labels to one-hot encoding
        labels_one_hot = F.one_hot(labels, num_classes=logits.size(1)).float()

        mse_loss = nn.MSELoss()
        loss = mse_loss(logits, labels_one_hot)

        _, predicted = torch.max(logits, dim=1)  # Shape: [batch_size]

        correct = (predicted == labels).sum().item()  # Count the number of matches

        

        # Softmax to logits to get probabilities
        # probs = F.softmax(logits, dim=1)

        # Log probabilities (for correct class and incorrect classes)
        # log_probs = F.log_softmax(logits, dim=1)

        # Log probability for the correct class
        # correct_class_log_probs = log_probs.gather(1, labels.view(-1, 1))
        
        # Correct class log-likelihood
        # correct_class_loss = -correct_class_log_probs

        # Penalty for incorrect classes
        # incorrect_class_penalty = torch.zeros_like(correct_class_loss)
        # incorrect_class_penalty = 0
        # Calculate the incorrect class penalties per class
        # for i in range(num_classes):
        #     incorrect_mask = (labels.unsqueeze(1) != i).float()  # Shape [batch_size, 1] vs [batch_size, 1] for broadcasting
        #     incorrect_class_penalty += torch.exp(-log_probs[:, i]) * incorrect_mask

        # Add the small constant factor 'e^(-j)' to the incorrect class penalty
        # incorrect_class_penalty += torch.exp(-self.j)

        # Combine the two loss terms
        # total_loss = correct_class_loss + torch.log(incorrect_class_penalty)

        # Average total loss over the batch
        # return total_loss.mean()

In [ ]:
cost = nn.CrossEntropyLoss()

# Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    correct = 0
    total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Tracking accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if (i+1) % len(train_loader) == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {accuracy:.2f}%')

Epoch [1/30], Step [938/938], Loss: 6.8133
Epoch [1/30], Training Accuracy: 9.75%
Epoch [2/30], Step [938/938], Loss: 6.8133
Epoch [2/30], Training Accuracy: 9.75%
Epoch [3/30], Step [938/938], Loss: 6.8133
Epoch [3/30], Training Accuracy: 9.75%
Epoch [4/30], Step [938/938], Loss: 6.8133
Epoch [4/30], Training Accuracy: 9.75%
Epoch [5/30], Step [938/938], Loss: 6.8133
Epoch [5/30], Training Accuracy: 9.75%
Epoch [6/30], Step [938/938], Loss: 6.8133
Epoch [6/30], Training Accuracy: 9.75%
Epoch [7/30], Step [938/938], Loss: 6.8133
Epoch [7/30], Training Accuracy: 9.75%
Epoch [8/30], Step [938/938], Loss: 6.8133
Epoch [8/30], Training Accuracy: 9.75%
Epoch [9/30], Step [938/938], Loss: 6.8133
Epoch [9/30], Training Accuracy: 9.75%
Epoch [10/30], Step [938/938], Loss: 6.8133
Epoch [10/30], Training Accuracy: 9.75%
Epoch [11/30], Step [938/938], Loss: 6.8133
Epoch [11/30], Training Accuracy: 9.75%
Epoch [12/30], Step [938/938], Loss: 6.8133
Epoch [12/30], Training Accuracy: 9.75%
Epoch [13/

In [40]:
# Check number of images in train and test folders
print(len(os.listdir('./data/train')))
print(len(os.listdir('./data/test')))

# Check number of labels in the label files
with open('./data/train_label.txt', 'r') as f:
    train_labels = f.readlines()
with open('./data/test_label.txt', 'r') as f:
    test_labels = f.readlines()

print(len(train_labels))  # Should match the number of images in the 'train' folder
print(len(test_labels))   # Should match the number of images in the 'test' folder

60000
10000
60000
10000
